In [1]:
import os
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import json
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
#from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report,roc_auc_score
#from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import utils

import gc

In [2]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

## Neural Network

In [3]:
BATCH_SIZE = 16
MAX_EPOCH = 20
SAVE_THRESHOLD = 0.6
SAVE_LOGITS = True

In [4]:
train_y.head()

,Unnamed: 0,target
0,2942719,0.0
1,4875524,0.0
2,6589819,0.0
3,1172060,1.0
4,2069395,0.0


In [5]:
train_y.target.iloc[3]

1.0

In [6]:
class myDataset(Dataset):
    def __init__(self, dfX, dfy):
        #'Initialization'
        self.dfX = dfX
        self.dfy = dfy
    def __len__(self):
        #'Denotes the total number of samples'
        return len(self.dfX)

    def __getitem__(self, index):
        #'Generates one sample of data'
        # Load data and get label
        X = self.dfX.iloc[index]
        X = torch.tensor(X)
        y = self.dfy.target.iloc[index]
        y = torch.tensor(y)
        return X, y

In [7]:
torch.tensor(train_X.iloc[1])


tensor([1.0000e+00, 4.8755e+06, 5.2240e+03, 1.4524e+05, 1.6000e+01, 6.0000e+00,
        8.0000e+00, 2.0062e+05, 3.7100e+02, 3.4892e+04, 7.4276e+04, 2.6024e+04,
        6.0000e+00, 3.0000e+00, 4.1000e+01, 1.0000e+00, 2.0000e+00, 4.9078e-01,
        5.0000e+03, 6.4750e+03])

In [8]:
#model
model = nn.Sequential(
    nn.Linear(20,40),
    nn.BatchNorm1d(40),
    nn.ReLU(),
    nn.Linear(40,60),
    nn.BatchNorm1d(60),
    nn.ReLU(),
    nn.Linear(60,60),
    nn.BatchNorm1d(60),
    nn.ReLU(),
    nn.Linear(60,1),
    nn.Sigmoid()
)

device, gpu_ids = utils.get_available_devices()
print(device,gpu_ids)
model = nn.DataParallel(model, gpu_ids)
model = model.to(device)

cuda:0 [0]


In [9]:
# Optimizer
#criterion = torch.nn.BCEWithLogitsLoss()   # default
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=L2_PENALTY)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4, min_lr=3e-5, verbose=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4, 
                                                       min_lr=1e-4, verbose=True)


In [10]:
def test(model, test_loader, criterion):
    model.eval();
    test_loss = 0.0
    true_ans_list = []
    preds_cat = []
    logits_list = []

    with torch.no_grad():
        for step, (features, targets) in enumerate(test_loader):
            features, targets = cuda(features), cuda(targets)
            logits = model(features)         
            loss = criterion(logits, targets)
            test_loss += loss.item()
            true_ans_list.append(targets)
            preds_cat.append(torch.sigmoid(logits))
            logits=logits.cpu().numpy()
            logits_list.append(logits)

        all_true_ans = torch.cat(true_ans_list)
        all_preds = torch.cat(preds_cat)
        all_preds = torch.ge(all_preds, 0.5)
        all_true_ans=all_true_ans.cpu()
        all_preds=all_preds.cpu()
        f1_eval = f1_score(all_true_ans, all_preds, average='weighted')
        f1_each = f1_score(all_true_ans, all_preds, average=None)
        # drop the last logits_list
        logits_list = logits_list[:-1]
        # print("list:", logits_list)
        all_logits = np.stack(logits_list)

    return all_logits, all_true_ans, all_preds


def validate(model, valid_loader, criterion):
    model.eval();
    test_loss = 0.0
    true_ans_list = []
    preds_cat = []

    with torch.no_grad():
        valid_iterator = valid_loader

        for step, (features, targets) in enumerate(valid_loader):
            features, targets = cuda(features), cuda(targets)
            logits = model(features)

            loss = criterion(logits, targets)
            test_loss += loss.item()
            true_ans_list.append(targets)
            preds_cat.append(torch.sigmoid(logits))

        all_true_ans = torch.cat(true_ans_list)
        all_preds = torch.cat(preds_cat)
        all_preds = torch.ge(all_preds, 0.5)
        all_true_ans=all_true_ans.cpu()
        all_preds=all_preds.cpu()
        # f1_eval = f1_score(all_true_ans, all_preds).item()
        f1_eval = f1_score(all_true_ans, all_preds, average='weighted')
        f1_each = f1_score(all_true_ans, all_preds, average=None)

    return test_loss / (step + 1), f1_eval, f1_each



In [11]:
def cuda(x):
    return x.cuda(non_blocking=True)

### NN - Draft & Debugging on tiny dataset

In [12]:
train_dataset_tiny = myDataset(train_tiny_X,train_tiny_y)
val_dataset_tiny = myDataset(val_tiny_X, val_tiny_y)

In [13]:
train_tiny_loader = DataLoader(train_dataset_tiny, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_tiny_loader = DataLoader(val_dataset_tiny, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [14]:
train_loss_history = []
train_epoch_losses = []
valid_losses = []
valid_f1s = []
best_model_f1 = 0.0
best_model_f1_each = []  # f1 for each label
best_model = None
best_model_ep = 0
num_ = len(os.listdir('output/')) + 1

for epoch in range(1, MAX_EPOCH + 1):
    #log.info('Starting epoch {}...'.format(epoch))
    print('starting Epoch:',epoch)
    model.train()
    total_loss = 0.0
    train_tqdm = tqdm(train_tiny_loader)
    train_loss_curr_epoch = []

    for step, (features, targets) in enumerate(train_tqdm):

        features = features.to(device=device) # move to device, e.g. GPU
        targets = targets.to(device=device, dtype=torch.long)

        features, targets = cuda(features), cuda(targets)
                
        optimizer.zero_grad()
        logits = model(features)  

        loss = criterion(logits, targets)            
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        total_loss += loss.item()

        if (step+1) % LOSS_HIST_ACCUM_FREQ == 0:
            tr_loss_ = total_loss / step
#             if (step+1) % (LOSS_HIST_ACCUM_FREQ*4) == 0:
#                 log.info('Train loss on iter {} : {}'.format((step+1), round(tr_loss_, 6)))

            train_loss_history.append(tr_loss_)
            train_loss_curr_epoch.append(tr_loss_)

    avg_epoch_loss = np.mean(train_loss_curr_epoch)
    #log.info('Mean train loss: {}'.format(round(avg_epoch_loss, 6)))
    print('mean train loss:', avg_epoch_loss)
    train_epoch_losses.append(avg_epoch_loss)

    print('Evaluating...')
    valid_loss, valid_f1, valid_f1_each = validate(model, val_tiny_loader, criterion)
    print('Validation loss:',(round(valid_loss, 6)))
    print('Validation F1: ',(round(valid_f1, 5)))
    valid_losses.append(valid_loss)

    valid_f1s.append(valid_f1)
    scheduler.step(valid_loss)

    # train_tqdm.set_postfix(epoch=epoch, loss=avg_epoch_loss)      

    if valid_f1 >= best_model_f1:
        best_model = model
        best_model_f1 = valid_f1
        best_model_ep = epoch
        best_model_f1_each = valid_f1_each

    if (epoch > 3) and (epoch % 4 == 0) and best_model_f1 > SAVE_THRESHOLD:
        #log.info('Saving best model ({})...'.format(num_))
        model_pt = 'output/best_model{}.pt'.format(num_)
        torch.save(best_model, model_pt)

    if (epoch > 3) and SAVE_RESULTS:
        results_dict = dict()
        results_dict['train_loss_history'] = train_loss_history
        results_dict['valid_loss'] = valid_losses
        results_dict['valid_f1'] = valid_f1s
        results_dict['best_valid_f1_each_label'] = best_model_f1_each.tolist()
        results_path = 'output/results{}.json'.format(num_)
        with open(results_path, 'w') as fp:
            json.dump(results_dict, fp, indent=4)

    if (epoch > 7) and (utils.check_early_stopping(valid_losses, patience=6, threshold=3e-3)):
        #log.info('Early stopping...')
        break

print()
log.info('Best F1 is {} on epoch {}'.format(round(best_model_f1, 5), best_model_ep))
print('Best F1_each is {} on epoch {}'.format(best_model_f1_each.tolist(), best_model_ep))


# Save model (remember to change model name)
if best_model_f1 > SAVE_THRESHOLD:
    #log.info('Saving best model...')
    torch.save(best_model, 'output/best_model{}.pt'.format(num_))

if SAVE_LOGITS:
    #log.info('Saving test set info...')
    test_logits, test_true_ans, test_preds = test(best_model, test_loader, criterion)
    # test_logits = test_logits.cpu().numpy()
    test_true_ans = test_true_ans.cpu().numpy()
    test_preds = test_preds.cpu().numpy()
    test_path = 'output/test_results{}.npz'.format(num_)
    np.savez(test_path, test_logits=test_logits, test_true_ans=test_true_ans, test_preds=test_preds)

# Load weights
if LOAD_MODEL:
    model_path = 'output/best_model3.pt'
    #log.info('Loading weights from {}...'.format(model_path))

    # model.load_state_dict(torch.load('output/saved/best_model1.pt'))
    model = torch.load(model_path)
    model.to(device)
    # model.eval()
    # valid_loss, valid_f1 = validate(model, test_loader, criterion, need_tqdm=False)

if SAVE_PLOT:
    xs = list(range(1, len(train_epoch_losses) + 1))
    plt.title('Loss')
    plt.plot(xs, train_epoch_losses, '-o', label = 'train')
    plt.plot(xs, valid_losses, '-o', label = 'valid')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.savefig('output/train_val_loss_dense{}.png'.format(num_))
    plt.show()
    plt.close()

    plt.title('F1')
    plt.plot(xs, valid_f1s, '-o')
    plt.ylabel('Valid F1')
    plt.xticks(xs);
    plt.xlabel('epoch');
    plt.savefig('output/valid_f1{}.png'.format(num_))
    plt.show()


  0%|          | 0/625 [00:00<?, ?it/s]

starting Epoch: 1


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorMath.cu:26

### NN - Full dataset w/o tunning

### NN - Tunning